<a href="https://colab.research.google.com/github/DivyaRustagi10/indic-ctm/blob/main/ZeroShot_Topic_Modeling_for_Indic_Languages_(IndicCTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  ZeroShot Topic Modeling for Indic Languages (IndicCTM)
*Previously Zero-shot Cross-Lingual Topic Modeling For Same Script Languages*
> Do conxtextualized TM tackle zero-shot cross-lingual topic modeling better on same script languages?

We use 4000 documents as training and consider randomly sampled 800 documents as the test set. We collect the 800 respective instances in Hindi (hi), Assamese (as), Gujarati (gu), Kannada (kn), Malayalam (ml), Marathi (mr), Oriya (or), Punjabi (pa), Tamil (ta), Telugu (te) and English (en).

First, we use IndicBERT to generate multilingual embeddings as the input of the model. Then we evaluate multilingual topic predictions on the multilingual abstracts in test set.

### Initial Setup

In [ ]:
# Install the contextualized topic model library
%%capture
!pip install -U contextualized_topic_models

In [ ]:
%%capture
!pip install pyldavis
!pip install wget
!pip install head
!nvidia-smi

# Setup Hindi for analysis
!pip install indic-nlp-library==0.81
!pip install stopwordsiso
#!pip install torch==1.13.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install inltk
!pip install regex
!pip install urduhack

In [ ]:
# Imports
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import  WhiteSpacePreprocessingStopwords
import pickle

In [ ]:
from inltk.inltk import setup

try:
  setup('hi')
except:
  pass

## Data

###**Building PMIndia Corpus**

Below contains the code for creating a parallel corpus from the website of the Indian Prime Minister (www.pmindia.gov.in). 

We combine each speech document into one, for every language. Datasets are downloaded from [Statistical Machine Translation](https://data.statmt.org/pmindia/v1/monolingual/).

*Downloading Dataset*

In [ ]:
# Imports
import urllib
import wget
import pandas as pd
import os

# Download PMIndia Datasets
FILES_DIR = os.getcwd() # REPLACE WITH YOUR DIRECTORY IF YOU PREFER DOWNLOADING IN SPECIFIC DIRECTORY
LINK = "https://drive.google.com/u/0/uc?id=1IqH2XQFw1XHPT2Sh_Yz3LnrMEVqS8oef&export=download"  # LINK TO PMINDIA FILE LOCATION
FILE_PATH = os.getcwd() + "/" + wget.download(LINK) # PATH TO DOWNLOADS

pmindia_list = pd.read_csv(FILE_PATH, sep = ",",  names = ["lang", "link"], engine = "python" )['link']
pmindia_list = [FILES_DIR + "/" + wget.download(link.strip(" ")) for link in pmindia_list]

In [ ]:
# Build directory to store speech files
%%bash
mkdir parallel_speeches
cd parallel_speeches

In [ ]:
# Files stored in content directory
# Data will need to be re-downloaded if a session closes
''' Following script will download parallel corpus into new directory named parallel_speeches.
Each folder in parallel_speeches contains pmindia speeches in indic language identified by their ISO code.

For e.g.
parallel_speeches/as/ contains speeches in Assamese.
parallel_speeches/hi contains speeches in Hindi.
'''
import tarfile
import os

FILES_DIR = os.getcwd() # REPLACE WITH DIRECTORY OF FILES
STORE_FILES_HERE = os.getcwd() + "/parallel_speeches" # STORE EXTRACTED FILES HERE

for fname in pmindia_list:
    tar = tarfile.open(fname, "r:gz") # unzip file
    tar.extractall(STORE_FILES_HERE)    
    foldername = fname[fname.rfind('.tgz')-2: fname.rfind('.tgz')] # get folder name
    os.rename(STORE_FILES_HERE + "/split", STORE_FILES_HERE + "/" + foldername) # rename default split folder to language name
    tar.close()

We'll only do this for the first time.


*Get Parallel Speeches*

In [ ]:
import glob # SEEK FILES FROM ABOVE FOR DOWNLOAD
FOLDERS_DIR = os.getcwd() + "/parallel_speeches"  # DIRECTORY FOR SAVING PARALLEL SPEECHES

# Stores list of speeches in dictionary keyed by ISO language name
SPEECHES_IN_LANGS = {languagefolder[-2:] : sorted(list(glob.glob(languagefolder + "/*.txt")))[3:] for languagefolder in glob.glob(FOLDERS_DIR + "/*")}

In [ ]:
# Get file names to find common test corpus (set of files common between all languages)
STRIP_INDEX = SPEECHES_IN_LANGS['as'][0].index('as') + len('as/')   # GET LANGUAGE ISO CODE

filenames = {}
for lang in SPEECHES_IN_LANGS.keys():
  temp = [item[STRIP_INDEX:] for item in SPEECHES_IN_LANGS[lang]]
  filenames[lang] = temp

*Combine Parallel Speeches Into Corpus*

In [ ]:
# Setting seed for reproducibility
import random
random.seed(210)

# Select 800 random corpus
sample_corpus = sorted(random.sample(list(set.intersection(*map(set, filenames.values()))), 800))
pd.DataFrame(sample_corpus, columns = ["Speech File Name"]).head()

,Speech File Name
0,15th-edition-of-pravasi-bharatiya-diwas-inaugu...
1,ambassador-ms-nikki-haley-united-states-perman...
2,anganwadi-workers-from-across-the-country-call...
3,asha-representatives-from-across-the-country-c...
4,beneficiaries-of-pradhan-mantri-mudra-yojana-s...


In [ ]:
# NORMALIZE HINDI TEXTS
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
import urduhack 

def normalize(sent, lang = 'hi'):  
  normalizer_factory = IndicNormalizerFactory()
  normalizer = normalizer_factory.get_normalizer(lang)
  normalized = normalizer.normalize(sent)
  return normalized

In [ ]:
# Combine selected 800 sample speeches into one document for each lang 
parallel_speeches = {}    # STORE TEST SPEECHES
train_speeches = {}       # STORE TRAIN SPEECHES

for lang, speeches in SPEECHES_IN_LANGS.items(): 
  each_lang = []         # list of speeches for one language at a time
  just_train = []
  
  for speech_file in speeches:                          # access list of files in speeches for one language at a time
    with open(speech_file, 'r') as speech:              # read file
      speech = " ".join([str(line) for line in speech]) # each speech file becomes one string
      
      if lang != 'en' and lang != 'ur':                 # normalize each speech per language
        normalize(speech, lang)

      if speech_file[STRIP_INDEX:] in sample_corpus:
        each_lang.append(speech)                        # append string version of speech file
      else:
        just_train.append(speech)                       # add to train set

  parallel_speeches[lang] = each_lang                   # add list of speeches for every language
  train_speeches[lang] = just_train                     # add to train set

train_speeches['hi'] = [normalize(list) for list in train_speeches['hi']]

*Split Parallel Corpus Into Test and Train*

In [ ]:
# Imports
import pandas as pd
from pprint import pprint
import re

# Selecting Train speeches
hindi_unprep = pd.DataFrame(list for list in train_speeches['hi'])
english_unprep = pd.DataFrame(list for list in train_speeches['en'])

# View each row as a speech
print(hindi_unprep[:5])
print(english_unprep[:5])

                                                   0
0  नागरिकों से स्वच्छाग्रही बनने और स्वच्छ भारत ब...
1  श्री सोमनाथ न्यास के न्यासियों की 116वीं बैठक ...
2  प्रधानमंत्री श्री नरेन्द्र मोदी की अध्यक्षता म...
3  प्रधानमंत्री श्री नरेन्द्र मोदी ने आज ही के दि...
4  · खूंटी की जिला अदालत में छत पर लगने वाले सौर ...
                                                   0
0  PM Calls upon citizens to become Swachhagrahis...
1  The first 10 months of Prime Minister Narendra...
2  BRICS in Africa: Collaboration for Inclusive G...
3  The 116th meeting of the trustees of Shri Somn...
4  Deendayal Upadhyaya Gram Jyoti Yojana\n The Un...


## Training Models

### Setup

In [ ]:
# Imports
from contextualized_topic_models.models.ctm import ZeroShotTM
import contextualized_topic_models.utils.data_preparation
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk
import pickle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords as stop_words
from gensim.utils import deaccent
import warnings
from inltk.inltk import remove_foreign_languages
import regex 
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize.indic_tokenize import trivial_tokenize

class WhiteSpacePreprocessingStopwords(WhiteSpacePreprocessingStopwords):
    """
    Overriden author's original code to keep accents during preprocessing.
    Provides a very simple preprocessing script that filters infrequent tokens from text
    """
    
    def tokenize_indic(self, sent, LANG = 'hi'):      
      normalizer_factory = IndicNormalizerFactory()
      normalizer = normalizer_factory.get_normalizer(LANG)
      normalized = normalizer.normalize(sent)
      tokenized = ' '.join(trivial_tokenize(normalized, LANG))
      return tokenized
    
    def custom_analyzer(self, text):
        """
        code source: https://stackoverflow.com/questions/60763030
                      having-an-issue-in-doing-count-vectorization-for-hindi-text
        """
        words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
        for w in words:
            yield w

    def preprocess(self):
        """
        Note that if after filtering some documents do not contain words we remove them. That is why we return also the
        list of unpreprocessed documents.
        :return: preprocessed documents, unpreprocessed documents and the vocabulary list
        """
        preprocessed_docs_tmp = self.documents
        preprocessed_docs_tmp = [doc.translate(
            str.maketrans(string.punctuation, ' ' * len(string.punctuation))) for doc in preprocessed_docs_tmp]
        if self.remove_numbers:
            preprocessed_docs_tmp = [doc.translate(str.maketrans("0123456789", ' ' * len("0123456789")))
                                     for doc in preprocessed_docs_tmp]
        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if len(w) > 0 and w not in self.stopwords])
                                 for doc in preprocessed_docs_tmp]
     
        preprocessed_docs_tmp = [self.tokenize_indic(sent) for sent in preprocessed_docs_tmp]
        
        # USE CUSTOM ANALYZER TO PRESERVE ACCENTS IN VECTORIZER
        vectorizer = CountVectorizer(analyzer = self.custom_analyzer, max_features=self.vocabulary_size, max_df=self.max_df)
        vectorizer.fit_transform(preprocessed_docs_tmp)
        temp_vocabulary = set(vectorizer.get_feature_names())

        preprocessed_docs_tmp = [' '.join([w for w in doc.split() if w in temp_vocabulary])
                                 for doc in preprocessed_docs_tmp]

        preprocessed_docs, unpreprocessed_docs, retained_indices = [], [], []
        for i, doc in enumerate(preprocessed_docs_tmp):
            if len(doc) > 0 and len(doc) >= self.min_words:
                preprocessed_docs.append(doc)
                unpreprocessed_docs.append(self.documents[i])
                retained_indices.append(i)

        vocabulary = list(set([item for doc in preprocessed_docs for item in doc.split()]))

        return preprocessed_docs, unpreprocessed_docs, vocabulary, retained_indices                                              

Above class method is taken from author's original code and modified to support preprocessing of Hindi documents. Additional helper methods have been added to support the updated preprocessing() method.

FIXED: Returns expected preprocessed hindi texts.

###**Hindi-Based ZeroshotTM**

*Preprocessing*

Why do we use the preprocessed text here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [ ]:
### HINDI ###
LANG_SELECTED = 'hi'

# We select 200 tokens per speech
NUM_TOKENS = 200

# Import Hindi Stopwords
import stopwordsiso as stopwords

# Run preprocessing script
documents = [line[:NUM_TOKENS].strip() for line in train_speeches[LANG_SELECTED]]
sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list = stopwords.stopwords(LANG_SELECTED))
preprocessed_documents, unpreprocessed_corpus, vocab, __ = sp.preprocess()

# Ensure same length for preprocessed and unpreprocessed
print(len(preprocessed_documents), len(unpreprocessed_corpus))

4003 4003


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#stopwords.stopwords('hi')

In [ ]:
from pprint import pprint
pprint(preprocessed_documents[:5])

['नागरिकों बनने स्वच्छ भारत बनाने आह्वान प्रधानमंत्री श्री नरेन्द्र मोदी '
 'महात्मा गांधी साल पूरे अवसर राष्ट्रीय राजधानी',
 'श्री वीं बैठक आज सम्पन्न हुई बैठक प्रधानमंत्री श्री नरेंद्र मोदी श्री आडवाणी '
 'श्री शाह श्री भाई पटेल श्री पी',
 'प्रधानमंत्री श्री नरेन्द्र मोदी अध्यक्षता मंत्रीमंडल बैठक दीनदयाल उपाध्याय '
 'ग्राम योजना प्रारंभ आज अनुमति दी गई योजना तहत ग्रामीण क्षेत्रों',
 'प्रधानमंत्री श्री नरेन्द्र मोदी आज दिन मुंबई आतंकी हमलों दौरान शहीद सलाम '
 'जिन्होंने जिंदगी दे',
 'जिला सौर ऊर्जा संयंत्र उद्घाटन किया प्रधानमंत्री मुद्रा योजना विशाल ऋण '
 'उद्घाटन लाभार्थियों ऋण दस्तावेज मुद्रा सौ']


We don't discard the non-preprocessed hindi texts, because we are going to use them as input for obtaining the **contextualized** document representations. We will now pass our files with preprocess and unpreprocessed data to our TopicModelDataPreparation object. This object takes care of creating the bag of words and obtains the contextualized BERT representations of documents. This operation allows us to create our training dataset.

Note: Here we use the contextualized model "ai4bharat/indic-bert", because we need a multilingual model for indic languages for performing cross-lingual predictions later.



*Training ZeroshotTM*

In [ ]:
from contextualized_topic_models.utils import data_preparation as dp
import numpy as np
from sentence_transformers import SentenceTransformer
import scipy.sparse
import warnings
from contextualized_topic_models.datasets.dataset import CTMDataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

class TopicModelDataPreparation(TopicModelDataPreparation):
      def custom_analyzer(self, text):
        """
        code source: https://stackoverflow.com/questions/60763030
                  having-an-issue-in-doing-count-vectorization-for-hindi-text
        """
        words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
        for w in words:
          yield w

      def fit(self, text_for_contextual, text_for_bow, labels=None, custom_embeddings=None):
        """
        This method fits the vectorizer and gets the embeddings from the contextual model
        :param text_for_contextual: list of unpreprocessed documents to generate the contextualized embeddings
        :param text_for_bow: list of preprocessed documents for creating the bag-of-words
        :param custom_embeddings: np.ndarray type object to use custom embeddings (optional).
        :param labels: list of labels associated with each document (optional).
        """

        if custom_embeddings is not None:
            assert len(text_for_contextual) == len(custom_embeddings)

            if text_for_bow is not None:
                assert len(custom_embeddings) == len(text_for_bow)

            if type(custom_embeddings).__module__ != 'numpy':
                raise TypeError("contextualized_embeddings must be a numpy.ndarray type object")

        if text_for_bow is not None:
            assert len(text_for_contextual) == len(text_for_bow)

        if self.contextualized_model is None and custom_embeddings is None:
            raise Exception("A contextualized model or contextualized embeddings must be defined")

        # TODO: this count vectorizer removes tokens that have len = 1, might be unexpected for the users
        self.vectorizer = CountVectorizer(analyzer = self.custom_analyzer)

        train_bow_embeddings = self.vectorizer.fit_transform(text_for_bow)

        # if the user is passing custom embeddings we don't need to create the embeddings using the model
        if custom_embeddings is None:
            train_contextualized_embeddings = dp.bert_embeddings_from_list(
                text_for_contextual, sbert_model_to_load=self.contextualized_model, max_seq_length=self.max_seq_length)
        else:
            train_contextualized_embeddings = custom_embeddings
        self.vocab = self.vectorizer.get_feature_names()
        self.id2token = {k: v for k, v in zip(range(0, len(self.vocab)), self.vocab)}

        if labels:
            self.label_encoder = OneHotEncoder()
            encoded_labels = self.label_encoder.fit_transform(np.array([labels]).reshape(-1, 1))
        else:
            encoded_labels = None
        return CTMDataset(
            X_contextual=train_contextualized_embeddings, X_bow=train_bow_embeddings,
            idx2token=self.id2token, labels=encoded_labels)

In [ ]:
# Load Indic Multilingual embeddings 
tp = TopicModelDataPreparation('ai4bharat/indic-bert')
tp.max_seq_length = 200

# Building training dataset
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/400M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ai4bharat_indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
%cd
!mkdir /content/models

/root


In [ ]:
# Train over 100 epochs
# Save model 
import joblib 

### HINDI : 25 TOPICS ###
z_ctm_25_HI = ZeroShotTM(bow_size=len(tp.vocab), n_components = 25, contextual_size=768, num_epochs=100)

z_ctm_25_HI.fit(training_dataset, n_samples = 30) # run the model
z_ctm_25_HI.save("./") # save the model
joblib.dump(z_ctm_25_HI, '/content/z_ctm_25_HI.pkl')


# ### HINDI : 50 TOPICS ###
z_ctm_50_HI = ZeroShotTM(bow_size=len(tp.vocab), n_components = 50, contextual_size=768, num_epochs=100)

z_ctm_50_HI.fit(training_dataset, n_samples = 30) # run the model
z_ctm_50_HI.save("./") # save the model
joblib.dump(z_ctm_50_HI, '/content/z_ctm_50_HI.pkl')

Epoch: [24/100]	 Seen Samples: [95232/400300]	Train Loss: 144.72921826762538	Time: 0:00:01.000268: : 24it [00:23,  1.01it/s]

### Topic Predictions

In [ ]:
# See topic predictions per speech doc
z_ctm_50_HI.get_topic_lists(5)[:4]

### PROBLEM
Predicted topics do not make sense in the train language - Hindi. Preprocessing works just fine. Something is getting lost in translation during model fit. 

* PATH 1: Topics are inferences from text. (NLI?)
* PATH 2: Topics are just words from within the text. (Extraction)
* PATH 3: Topics are lemmetized versions of preprocessed tokens. 


> *What is topic modeling?*

Topic modeling is an unsupervised machine learning technique that’s capable of scanning a set of documents, detecting word and phrase patterns within them, and automatically clustering word groups and similar expressions that best characterize a set of documents.

src: Google

In [ ]:
# See topic predictions per speech doc
z_ctm_25_HI.get_topic_lists(5)[:4]

In [ ]:
# See topic predictions per speech doc
z_ctm_50_HI.get_topic_lists(5)[:4]

In [ ]:
hi_topics_25 = open("hi_topics_25.csv","w")
hi_topics_50 = open("hi_topics_50.csv","w")

import csv
wr1 = csv.writer(hi_topics_25)
wr1.writerows(z_ctm_25_HI.get_topic_lists(5))

wr = csv.writer(hi_topics_50)
wr.writerows(z_ctm_50_HI.get_topic_lists(5))

**Topic Results for Hindi ZeroShot TM translated to English**

In [ ]:
# %cd ..
# %cd ..
# # clone the repo for running evaluation
# !git clone https://github.com/AI4Bharat/indicTrans.git
# %cd indicTrans
# # clone requirements repositories
# !git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
# !git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
# !git clone https://github.com/rsennrich/subword-nmt.git

In [ ]:
# %cd ..

In [ ]:
# # Install the necessary libraries
# !pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# ! pip install mosestokenizer subword-nmt
# # Install fairseq from source
# !git clone https://github.com/pytorch/fairseq.git

In [ ]:
# %cd fairseq

In [ ]:
# !pip install -U setuptools

In [ ]:
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
!pip install xformers

In [ ]:
# %cd ..
# # add fairseq folder to python path
# import os
# os.environ['PYTHONPATH'] += ":/content/fairseq/"
# # sanity check to see if fairseq is installed
# from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

# # downloading the indic-en model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
# !unzip indic-en.zip

In [ ]:
# %cd indicTrans
# from indicTrans.inference.engine import Model
# indic2en_model = Model(expdir='../indic-en')

In [ ]:
# # 25 TOPICS
# indic2en_model.batch_translate(z_ctm_25_HI.get_topic_lists(5)[:4], 'hi', 'en')    

In [ ]:
# # 50 TOPICS
# indic2en_model.batch_translate(z_ctm_50_HI.get_topic_lists(5)[:4], 'hi', 'en')

In [ ]:
pprint(preprocessed_documents[:4])

In [ ]:
# Get NPMI Coherence
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
texts = [doc.split() for doc in preprocessed_documents] # load text for NPMI

### 25 TOPICS ###
npmi_HI = CoherenceNPMI(texts=texts, topics=z_ctm_25_HI.get_topic_lists(25))
print(npmi_HI.score())

### 50 TOPICS ###
npmi_50_HI = CoherenceNPMI(texts=texts, topics=z_ctm_50_HI.get_topic_lists(50))
print(npmi_50_HI.score())

# Store NPMI scores
zeroshotNPMI_HI = [npmi_HI.score(), npmi_50_HI.score()]

###**English-Based ZeroshotTM**

*Preprocessing*

Why do we use the preprocessed text here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

In [ ]:
### ENGLISH ###
LANG_SELECTED = 'en'

# We select 200 tokens per speech
NUM_TOKENS = 200

# Download English Stopwords
nltk.download('stopwords')

# Run preprocessing script
documents = [line[:NUM_TOKENS].strip() for line in train_speeches[LANG_SELECTED]]

import nltk
nltk.download('words')
# preprocessed_documents
words = set(nltk.corpus.words.words())

documents = ["".join([word for word in doc if word.lower() in words or not word.isalpha()]) for doc in documents]

sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab, __ = sp.preprocess()

# Ensure same length for preprocessed and unpreprocessed
print(len(preprocessed_documents), len(unpreprocessed_corpus))

We don't discard the non-preprocessed english texts, because we are going to use them as input for obtaining the **contextualized** document representations. We will now pass our files with preprocess and unpreprocessed data to our TopicModelDataPreparation object. This object takes care of creating the bag of words and obtains the contextualized BERT representations of documents. This operation allows us to create our training dataset.

Note: Here we use the contextualized model "ai4bharat/indic-bert", because we need a multilingual model for indic languages for performing cross-lingual predictions later.



In [ ]:
# Building training dataset
tp = TopicModelDataPreparation("sentence-transformers/bert-base-nli-mean-tokens")
en_training = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

*Training ZeroshotTM*

In [ ]:
# Train over 100 epochs

### ENGLISH : 25 TOPICS ###
z_ctm_25_EN = ZeroShotTM(bow_size=len(tp.vocab), n_components = 25, contextual_size=768, num_epochs=100)
z_ctm_25_EN.fit(en_training, n_samples=30) # run the model
z_ctm_25_EN.save("./") # save the model
joblib.dump(z_ctm_25_EN, '/content/z_ctm_25_EN.pkl')

### ENGLISH : 50 TOPICS ###
z_ctm_50_EN = ZeroShotTM(bow_size=len(tp.vocab), n_components = 50, contextual_size=768, num_epochs=100)
z_ctm_50_EN.fit(en_training, n_samples=30) # run the model
z_ctm_50_EN.save("./") # save the model
joblib.dump(z_ctm_50_EN, '/content/z_ctm_50_EN.pkl')

In [ ]:
#save model 
import joblib 
joblib.dump(z_ctm_50_EN, '../content/z_ctm_50_EN.pkl')

### Topic Prediction Examples

In [ ]:
# See topic predictions per speech doc
z_ctm_25_EN.get_topic_lists(5)[:4]

In [ ]:
# See topic predictions per speech doc
z_ctm_50_EN.get_topic_lists(5)[:4]

# Coherence Results

In [ ]:
# Get NPMI Coherence
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
texts = [doc.split() for doc in preprocessed_documents] # load text for NPMI

### 25 TOPICS ###-
npmi_EN = CoherenceNPMI(texts=texts, topics=z_ctm_25_EN.get_topic_lists(25))
print(npmi_EN.score())

### 50 TOPICS ###
npmi_50_EN = CoherenceNPMI(texts=texts, topics=z_ctm_50_EN.get_topic_lists(50))
print(npmi_50_EN.score())

# Store NPMI scores
zeroshotNPMI_EN = [npmi_EN.score(), npmi_50_EN.score()]

In [ ]:
# SHOW RESULTS
NPMI = {"ZeroShotTM for Hindi" : zeroshotNPMI_HI,
        "ZeroShotTM for English" : zeroshotNPMI_EN}

npmi = pd.DataFrame.from_dict(NPMI, orient='index')
print("NPMI Coherences")
npmi.set_axis(["t(25)", "t(50)"], axis = 1)

# FastAPI Setup

**What is FastApi?**

FastAPI offers automatic docs generation functionality, authentication, data validation via pydantic models.

**Colab Code**

It is a Python package that allows you to start a code server right from your Colab notebooks without setting up anything locally on your system. 
It can be used to start a VS Code environment, Jupyter Lab server, or tunnel the FastAPI server to the web, all in the colab notebook. 

This can be a great plus point for the enthusiast coders who train their models on the cloud and now want to share their findings with the world in the form of APIs. 

Python 3.11 instructions: https://peps.python.org/pep-0681/

In [ ]:
# %%capture
# !pip install fastapi
# !pip install colabcode
# !pip install pyngrok

In [ ]:
# from pydantic import BaseModel

# # Create data class for input for data validation
# class Document(BaseModel):
#   input_text: str
#   class Config:
#     schema_extra = {
#         "example": {
#                 "input_text": 
#                 "ਗੁਜਰਾਤ ਦੇ 50 ਮਹਿਲਾ ਮੋਟਰਸਾਈਕਲ ਸਵਾਰਾਂ ਦੇ ਗਰੁੱਪ ‘ਬਾਈਕਿੰਗ ਕਵੀਨਜ਼’ ਨੇ ਅੱਜ ਇੱਥੇ ਪ੍ਰਧਾਨ ਮੰਤਰੀ, ਸ਼੍ਰੀ ਨਰੇਂਦਰ ਮੋਦੀ ਨਾਲ ਮੁਲਾਕਾਤ ਕੀਤੀ।"\
#                 "ਗਰੁੱਪ ਦਾ ਕਹਿਣਾ ਹੈ ਕਿ ਉਨ੍ਹਾਂ ਨੇ 13 ਰਾਜਾਂ / ਕੇਂਦਰ ਸ਼ਾਸਤ ਪ੍ਰਦੇਸ਼ਾਂ ’ਚ 10,000 ਕਿਲੋਮੀਟਰ ਦਾ ਸਫਰ ਤੈਅ ਕਰਦਿਆਂ ਰਾਹ ਵਿੱਚ ਬੇਟੀ ਬਚਾਓ, ਬੇਟੀ ਪੜ੍ਹਾਓ ਅਤੇ ਸਵੱਛ ਭਾਰਤ"\
#                 "ਵਰਗੇ ਕਈ ਸਮਾਜਿਕ ਮੁੱਦਿਆਂ ’ਤੇ ਲੋਕਾਂ ਨਾਲ ਚਰਚਾ ਕੀਤੀ।"\
#                 "ਉਨ੍ਹਾਂ ਨੇ 15 ਅਗਸਤ 2017 ਨੂੰ ਲੱਦਾਖ ਦੇ ਖਰਦੁੰਗਲਾ ਵਿੱਖੇ ਤਿਰੰਗਾ ਲਹਿਰਾਇਆ।"\
#                 "ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਨੇ ਉਨ੍ਹਾਂ ਦੇ ਯਤਨਾਂ ਦੀ ਪ੍ਰਸ਼ੰਸਾ ਕੀਤੀ ਅਤੇ ਉਨ੍ਹਾਂ ਦੇ ਚੰਗੇ ਭਵਿੱਖ ਦੀ ਕਾਮਨਾ ਕੀਤੀ।"
#             }
#         }

In [ ]:
# from starlette.responses import PlainTextResponse
# # Create endpoint for serving requests using endpoint function
# from fastapi import FastAPI, Response
# from fastapi.responses import UJSONResponse
# import joblib

# app = FastAPI()


# @app.on_event("startup")
# def load_model():
#   global hindi_model_25
#   global hindi_model_50
#   global tp
#   # Load Indic Multilingual embeddings
#   tp = TopicModelDataPreparation('ai4bharat/indic-bert')
#   tp.max_seq_length = 200

#   file = open("/content/models/z_ctm_25_HI.pkl", "rb")
#   hindi_model_25 = joblib.load(file)
#   hindi_model_50 = joblib.load(open("/content/models/z_ctm_50_HI.pkl", "rb"))
#   file.close()

# @app.get('/')
# def index():
#   return {'message': 'This is the homepage of the API '}

# @app.post('/predict')
# def get_topic_predictions(data: Document):
#   received = data.dict()
#   text = list(received['input_text'])
#   input_text = tp.transform(text)

#   document_index = 1
#   models = {'model_25': hindi_model_25, 'model_50': hindi_model_50}
#   output = {}
#   for name, model in models.items():
#     pred_name = model.get_thetas(input_text, n_samples = 10)
#     topic_number = np.argmax(pred_name[1]) # get the topic id of the first document
#     predicted_topics = model.get_topic_lists()[topic_number] 
#     output['prediction_' + name] = predicted_topics

#   return output

In [ ]:
# %%bash 
# ngrok authtoken 2HjWU6F1LiLRalwg0UqTl8iPD24_Pun7DFWSgABowbjkBSCj

In [ ]:
# # Spin Colab server using ColabCode
# from colabcode import ColabCode
# server = ColabCode(port=10000, code=False)
# server.run_app(app=app)

# Predictions and Evaluation
###**Unseen Multilingual  Corpora Predictions**

*Languages*

* Assamese - as
* Bengali - bn
* English - en
* Gujarati - gu
* Hindi - hi
* Kannada - kn
* Malayalam - ml
* Marathi - mr
* Oriya - or
* Punjabi - pa
* Tamil - ta
* Telugu - te

In [ ]:
# Convert test files into test datasets
as_testset = tp.transform(parallel_speeches['as'])
bn_testset = tp.transform(parallel_speeches['bn'])
en_testset = tp.transform(parallel_speeches['en'])
gu_testset = tp.transform(parallel_speeches['gu'])
hi_testset = tp.transform(parallel_speeches['hi'])
kn_testset = tp.transform(parallel_speeches['kn'])
ml_testset = tp.transform(parallel_speeches['ml'])
mr_testset = tp.transform(parallel_speeches['mr'])
or_testset = tp.transform(parallel_speeches['or'])
pa_testset = tp.transform(parallel_speeches['pa'])
ta_testset = tp.transform(parallel_speeches['ta'])
te_testset = tp.transform(parallel_speeches['te'])

###**Topic Predictions**

*Hindi*

In [ ]:
### HINDI : 25 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_25_HI.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_25_HI.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_25_HI.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_25_HI.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_25_HI.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_25_HI.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_25_HI.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_25_HI.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_25_HI.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_25_HI.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_25_HI.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_25_HI.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_25_HI = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

In [ ]:
### HINDI : 50 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_50_HI.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_50_HI.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_50_HI.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_50_HI.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_50_HI.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_50_HI.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_50_HI.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_50_HI.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_50_HI.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_50_HI.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_50_HI.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_50_HI.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_50_HI = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

*English*

In [ ]:
### ENGLISH : 25 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_25_EN.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_25_EN.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_25_EN.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_25_EN.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_25_EN.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_25_EN.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_25_EN.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_25_EN.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_25_EN.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_25_EN.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_25_EN.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_25_EN.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_25_EN = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

In [ ]:
### ENGLISH : 50 TOPIC PREDICTIONS ### 
as_topics_predictions = z_ctm_50_EN.get_thetas(as_testset, n_samples=100) # get all the topic predictions
bn_topics_predictions = z_ctm_50_EN.get_thetas(bn_testset, n_samples=100) # get all the topic predictions
en_topics_predictions = z_ctm_50_EN.get_thetas(en_testset, n_samples=100) # get all the topic predictions
gu_topics_predictions = z_ctm_50_EN.get_thetas(gu_testset, n_samples=100) # get all the topic predictions
hi_topics_predictions = z_ctm_50_EN.get_thetas(hi_testset, n_samples=100) # get all the topic predictions
kn_topics_predictions = z_ctm_50_EN.get_thetas(kn_testset, n_samples=100) # get all the topic predictions
ml_topics_predictions = z_ctm_50_EN.get_thetas(ml_testset, n_samples=100) # get all the topic predictions
mr_topics_predictions = z_ctm_50_EN.get_thetas(mr_testset, n_samples=100) # get all the topic predictions
or_topics_predictions = z_ctm_50_EN.get_thetas(or_testset, n_samples=100) # get all the topic predictions
pa_topics_predictions = z_ctm_50_EN.get_thetas(pa_testset, n_samples=100) # get all the topic predictions
ta_topics_predictions = z_ctm_50_EN.get_thetas(ta_testset, n_samples=100) # get all the topic predictions
te_topics_predictions = z_ctm_50_EN.get_thetas(te_testset, n_samples=100) # get all the topic predictions

topics_50_EN = {'as': as_topics_predictions, 'bn': bn_topics_predictions, 
             'en': en_topics_predictions, 'gu': gu_topics_predictions,
             'hi': hi_topics_predictions, 'kn': kn_topics_predictions,
             'ml': ml_topics_predictions, 'mr': mr_topics_predictions,
             'or': or_topics_predictions, 'pa': pa_topics_predictions,
             'ta': ta_topics_predictions, 'te': te_topics_predictions}

### **Quantitative Evaluation**

In [ ]:
# Import metrics
from contextualized_topic_models.evaluation.measures import Matches, KLDivergence, CentroidDistance
import warnings
warnings.filterwarnings('ignore')

1. **Matches**

> Matches is the % of times the predicted topic for the non-English test document is the same as for the respective test document in English. The higher the scores, the better.

*Hindi*

In [ ]:
# HINDI : Matches for 25 topics
hi_as_matches = Matches(topics_25_HI['hi'], topics_25_HI['as'])
hi_bn_matches = Matches(topics_25_HI['hi'], topics_25_HI['bn'])
hi_en_matches = Matches(topics_25_HI['hi'], topics_25_HI['en'])
hi_gu_matches = Matches(topics_25_HI['hi'], topics_25_HI['gu'])
hi_kn_matches = Matches(topics_25_HI['hi'], topics_25_HI['kn'])
hi_ml_matches = Matches(topics_25_HI['hi'], topics_25_HI['ml'])
hi_mr_matches = Matches(topics_25_HI['hi'], topics_25_HI['mr'])
hi_or_matches = Matches(topics_25_HI['hi'], topics_25_HI['or'])
hi_pa_matches = Matches(topics_25_HI['hi'], topics_25_HI['pa'])
hi_ta_matches = Matches(topics_25_HI['hi'], topics_25_HI['ta'])
hi_te_matches = Matches(topics_25_HI['hi'], topics_25_HI['te'])


matches_25_HI = {'as': hi_as_matches.score(), 'bn': hi_bn_matches.score(), 
             'en': hi_en_matches.score(), 'gu': hi_gu_matches.score(),
             'kn': hi_kn_matches.score(),
             'ml': hi_ml_matches.score(), 'mr': hi_mr_matches.score(),
             'or': hi_or_matches.score(), 'pa': hi_pa_matches.score(),
             'ta': hi_ta_matches.score(), 'te': hi_te_matches.score()}
matches_25_HI

In [ ]:
# Average Matches for 25 topics
average_matches_25_HI = sum(matches_25_HI.values())/len(matches_25_HI.keys())

In [ ]:
# HINDI : Matches for 50 topics
hi_as_matches = Matches(topics_50_HI['hi'], topics_50_HI['as'])
hi_bn_matches = Matches(topics_50_HI['hi'], topics_50_HI['bn'])
hi_en_matches = Matches(topics_50_HI['hi'], topics_50_HI['en'])
hi_gu_matches = Matches(topics_50_HI['hi'], topics_50_HI['gu'])
hi_kn_matches = Matches(topics_50_HI['hi'], topics_50_HI['kn'])
hi_ml_matches = Matches(topics_50_HI['hi'], topics_50_HI['ml'])
hi_mr_matches = Matches(topics_50_HI['hi'], topics_50_HI['mr'])
hi_or_matches = Matches(topics_50_HI['hi'], topics_50_HI['or'])
hi_pa_matches = Matches(topics_50_HI['hi'], topics_50_HI['pa'])
hi_ta_matches = Matches(topics_50_HI['hi'], topics_50_HI['ta'])
hi_te_matches = Matches(topics_50_HI['hi'], topics_50_HI['te'])


matches_50_HI = {'as': hi_as_matches.score(), 'bn': hi_bn_matches.score(), 
             'en': hi_en_matches.score(), 'gu': hi_gu_matches.score(),
             'kn': hi_kn_matches.score(),
             'ml': hi_ml_matches.score(), 'mr': hi_mr_matches.score(),
             'or': hi_or_matches.score(), 'pa': hi_pa_matches.score(),
             'ta': hi_ta_matches.score(), 'te': hi_te_matches.score()}
matches_50_HI

In [ ]:
# Average Matches for 50 topics
average_matches_50_HI = sum(matches_50_HI.values())/len(matches_50_HI.keys())

*English*

In [ ]:
# ENGLISH : Matches for 25 topics
en_as_matches = Matches(topics_25_EN['en'], topics_25_EN['as'])
en_bn_matches = Matches(topics_25_EN['en'], topics_25_EN['bn'])
en_hi_matches = Matches(topics_25_EN['en'], topics_25_EN['hi'])
en_gu_matches = Matches(topics_25_EN['en'], topics_25_EN['gu'])
en_kn_matches = Matches(topics_25_EN['en'], topics_25_EN['kn'])
en_ml_matches = Matches(topics_25_EN['en'], topics_25_EN['ml'])
en_mr_matches = Matches(topics_25_EN['en'], topics_25_EN['mr'])
en_or_matches = Matches(topics_25_EN['en'], topics_25_EN['or'])
en_pa_matches = Matches(topics_25_EN['en'], topics_25_EN['pa'])
en_ta_matches = Matches(topics_25_EN['en'], topics_25_EN['ta'])
en_te_matches = Matches(topics_25_EN['en'], topics_25_EN['te'])


matches_25_EN = {'as': en_as_matches.score(), 'bn': en_bn_matches.score(), 
             'en': en_hi_matches.score(), 'gu': en_gu_matches.score(),
             'kn': en_kn_matches.score(),
             'ml': en_ml_matches.score(), 'mr': en_mr_matches.score(),
             'or': en_or_matches.score(), 'pa': en_pa_matches.score(),
             'ta': en_ta_matches.score(), 'te': en_te_matches.score()}
matches_25_EN

In [ ]:
# Average Matches for 25 topics
average_matches_25_EN = sum(matches_25_EN.values())/len(matches_25_EN.keys())

In [ ]:
# ENGLISH : Matches for 50 topics
en_as_matches = Matches(topics_50_EN['en'], topics_50_EN['as'])
en_bn_matches = Matches(topics_50_EN['en'], topics_50_EN['bn'])
en_en_matches = Matches(topics_50_EN['en'], topics_50_EN['hi'])
en_gu_matches = Matches(topics_50_EN['en'], topics_50_EN['gu'])
en_kn_matches = Matches(topics_50_EN['en'], topics_50_EN['kn'])
en_ml_matches = Matches(topics_50_EN['en'], topics_50_EN['ml'])
en_mr_matches = Matches(topics_50_EN['en'], topics_50_EN['mr'])
en_or_matches = Matches(topics_50_EN['en'], topics_50_EN['or'])
en_pa_matches = Matches(topics_50_EN['en'], topics_50_EN['pa'])
en_ta_matches = Matches(topics_50_EN['en'], topics_50_EN['ta'])
en_te_matches = Matches(topics_50_EN['en'], topics_50_EN['te'])


matches_50_EN = {'as': en_as_matches.score(), 'bn': en_bn_matches.score(), 
             'en': en_hi_matches.score(), 'gu': en_gu_matches.score(),
             'kn': en_kn_matches.score(),
             'ml': en_ml_matches.score(), 'mr': en_mr_matches.score(),
             'or': en_or_matches.score(), 'pa': en_pa_matches.score(),
             'ta': en_ta_matches.score(), 'te': en_te_matches.score()}
matches_50_EN

In [ ]:
# Average Matches for 50 topics
average_matches_50_EN = sum(matches_50_EN.values())/len(matches_50_EN.keys())

2. **Distributional Similarity**
> Compute the KL divergence between the predicted topic distribution on the test document and the same test document in English. Lower scores are better, indicating that the distributions do not differ by much.

*Hindi*

In [ ]:
# HINDI : KL Divergence for 25 topics
hi_as_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['as'])
hi_bn_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['bn'])
hi_en_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['en'])
hi_gu_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['gu'])
hi_kn_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['kn'])
hi_ml_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['ml'])
hi_mr_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['mr'])
hi_or_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['or'])
hi_pa_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['pa'])
hi_ta_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['ta'])
hi_te_kl = KLDivergence(topics_25_HI['hi'], topics_25_HI['te'])

kl_divergence_25_HI = {'as': hi_as_kl.score(), 'bn': hi_bn_kl.score(), 
             'en': hi_en_kl.score(), 'gu': hi_gu_kl.score(),
             'kn': hi_kn_kl.score(),
             'ml': hi_ml_kl.score(), 'mr': hi_mr_kl.score(),
             'or': hi_or_kl.score(), 'pa': hi_pa_kl.score(),
             'ta': hi_ta_kl.score(), 'te': hi_te_kl.score()}

kl_divergence_25_HI

In [ ]:
# Average KLD for 25 topics
average_kl_divergence_25_HI = sum(kl_divergence_25_HI.values())/len(kl_divergence_25_HI.keys())

In [ ]:
# HINDI : KL Divergence for 50 topics
hi_as_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['as'])
hi_bn_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['bn'])
hi_en_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['en'])
hi_gu_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['gu'])
hi_kn_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['kn'])
hi_ml_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['ml'])
hi_mr_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['mr'])
hi_or_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['or'])
hi_pa_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['pa'])
hi_ta_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['ta'])
hi_te_kl = KLDivergence(topics_50_HI['hi'], topics_50_HI['te'])

kl_divergence_50_HI = {'as': hi_as_kl.score(), 'bn': hi_bn_kl.score(), 
             'en': hi_en_kl.score(), 'gu': hi_gu_kl.score(),
             'kn': hi_kn_kl.score(),
             'ml': hi_ml_kl.score(), 'mr': hi_mr_kl.score(),
             'or': hi_or_kl.score(), 'pa': hi_pa_kl.score(),
             'ta': hi_ta_kl.score(), 'te': hi_te_kl.score()}

kl_divergence_50_HI

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_50_HI = sum(kl_divergence_50_HI.values())/len(kl_divergence_50_HI.keys())

*English*

In [ ]:
# ENGLISH : KL Divergence for 25 topics
en_as_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['as'])
en_bn_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['bn'])
en_hi_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['hi'])
en_gu_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['gu'])
en_kn_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['kn'])
en_ml_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['ml'])
en_mr_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['mr'])
en_or_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['or'])
en_pa_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['pa'])
en_ta_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['ta'])
en_te_kl = KLDivergence(topics_25_EN['en'], topics_25_EN['te'])

kl_divergence_25_EN = {'as': en_as_kl.score(), 'bn': en_bn_kl.score(), 
             'hi': en_hi_kl.score(), 'gu': en_gu_kl.score(),
             'kn': en_kn_kl.score(),
             'ml': en_ml_kl.score(), 'mr': en_mr_kl.score(),
             'or': en_or_kl.score(), 'pa': en_pa_kl.score(),
             'ta': en_ta_kl.score(), 'te': en_te_kl.score()}

kl_divergence_25_EN

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_25_EN = sum(kl_divergence_25_EN.values())/len(kl_divergence_25_EN.keys())

In [ ]:
# ENGLISH : KL Divergence for 50 topics
en_as_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['as'])
en_bn_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['bn'])
en_hi_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['hi'])
en_gu_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['gu'])
en_kn_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['kn'])
en_ml_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['ml'])
en_mr_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['mr'])
en_or_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['or'])
en_pa_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['pa'])
en_ta_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['ta'])
en_te_kl = KLDivergence(topics_50_EN['en'], topics_50_EN['te'])

kl_divergence_50_EN = {'as': en_as_kl.score(), 'bn': en_bn_kl.score(), 
             'hi': en_hi_kl.score(), 'gu': en_gu_kl.score(),
             'kn': en_kn_kl.score(),
             'ml': en_ml_kl.score(), 'mr': en_mr_kl.score(),
             'or': en_or_kl.score(), 'pa': en_pa_kl.score(),
             'ta': en_ta_kl.score(), 'te': en_te_kl.score()}

kl_divergence_50_EN

In [ ]:
# Average KLD for 50 topics
average_kl_divergence_50_EN = sum(kl_divergence_50_EN.values())/len(kl_divergence_50_EN.keys())

3. **Centroid Embeddings**
> To also account for similar but not exactly equal topic predictions, we compute the centroid embeddings of the 5 words describing the predicted topic for both English and non-English documents. Then we compute the cosine similarity between those two centroids (CD).

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import KeyedVectors
import gensim.downloader as api
from scipy.spatial.distance import cosine
import abc
import numpy as np

class CD(CentroidDistance):
    """Override author's function to upgrade compatibility with Gensim 4.0.0.
    See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4."""

    def get_centroid(self, word_list):
        vector_list = []
        for word in word_list:
            if word in self.wv:   # changed from self.wv.vocab to self.wv as in Gensim 4.0.0
                vector_list.append(self.wv.get_vector(word))
        vec = sum(vector_list)
        return vec / np.linalg.norm(vec)

*Hindi*

In [ ]:
# HINDI : Centroid Embeddings for 25 topics
cd_25_HI = {}

for key in topics_25_HI.keys():
  if key == 'hi':
    continue
  topic = topics_25_HI[key]
  cd = CD(doc_distribution_original_language = topics_25_HI['hi'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_25_HI.get_topic_lists(25),
          topk = 5)
  
  cd_25_HI[key] = cd.score()

cd_25_HI

In [ ]:
# Average KLD for 25 topics
average_cd_25_HI = sum(cd_25_HI.values())/len(cd_25_HI.keys())

In [ ]:
# HINDI : Centroid Embeddings for 50 topics
cd_50_HI = {}

for key in topics_50_HI.keys():
  if key == 'hi':
    continue
  topic = topics_50_HI[key]
  cd = CD(doc_distribution_original_language = topics_50_HI['hi'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_50_HI.get_topic_lists(50),
          topk = 5)
  
  cd_50_HI[key] = cd.score()
  cd = None

cd_50_HI

*English*

In [ ]:
# Average KLD for 25 topics
average_cd_25_HI = sum(cd_25_HI.values())/len(cd_25_HI.keys())

In [ ]:
# ENGLISH : Centroid Embeddings for 25 topics
cd_25_EN = {}

for key in topics_25_EN.keys():
  if key == 'en':
    continue
  topic = topics_25_EN[key]
  cd = CD(doc_distribution_original_language = topics_25_EN['en'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_25_EN.get_topic_lists(25),
          topk = 5)
  
  cd_25_EN[key] = cd.score()

cd_25_EN

In [ ]:
# ENGLISH : Centroid Embeddings for 50 topics
cd_50_EN = {}

for key in topics_50_EN.keys():
  if key == 'hi':
    continue
  topic = topics_50_EN[key]
  cd = CD(doc_distribution_original_language = topics_50_EN['en'], 
          doc_distribution_unseen_language = topic, 
          topics = z_ctm_50_EN.get_topic_lists(50),
          topk = 5)
  
  cd_50_EN[key] = cd.score()
  cd = None

cd_50_EN

In [ ]:
# Store Metrics
metrics = {
          "Hindi" : [{
                    "Mat25": matches_25_HI,
                    "KL25": kl_divergence_25_HI, 
                    "CD25": cd_25_HI, 
                    "Mat50": matches_50_HI, 
                    "KL50": kl_divergence_50_HI,
                    "CD50": cd_50_HI
                    }],

          "English": [{
                    "Mat25": matches_25_EN,
                    "KL25": kl_divergence_25_EN, 
                    "CD25": cd_25_EN, 
                    "Mat50": matches_50_EN, 
                    "KL50": kl_divergence_50_EN,
                    "CD50": cd_50_EN
                    }]
          }
with open("metrics_samescript.txt", 'wb') as F:
  pickle.dump(metrics, F)

metrics

# Evaluation Results

In [ ]:
# Show results
metrics = {"Hindi" : 
           
           {"Mat25": matches_25_HI,
           "KL25": kl_divergence_25_HI, 
           "CD25": cd_25_HI, 
           "Mat50": matches_50_HI, 
           "KL50": kl_divergence_50_HI,
           "CD50": cd_50_HI},
           
           "English" : 
           {"Mat25": matches_25_EN,
           "KL25": kl_divergence_25_EN, 
           "CD25": cd_25_EN, 
           "Mat50": matches_50_EN, 
           "KL50": kl_divergence_50_EN,
           "CD50": cd_50_EN}
           }

metrics = pd.DataFrame.from_dict(metrics, orient='columns') 
print("Match, KL, and Centroid Similarity for 25 and 50 topics on various languages on PMIndia Corpus")
metrics